In [ ]:
from scipy.stats import binom
import math
from texttable import Texttable

Notations

* `f` - presumed number of malicious nodes
* `c` - expected number of nodes qualified for backers
* `N` - total number of nodes

`F` computes a probability of a committee having at least `d` backers completely controlled by malicious nodes

In [38]:
def F(d, c, f, N):
    x = 0
    p = c / N
    for i in range(d, f):
        x += binom.pmf(i, f, p)
    return x

ds = [1,2,3,4,5,6]
f, c, N = 21, 8, 101
t = Texttable()
t.set_cols_dtype(['i','a'])
t.add_row(['d', 'F'])
for d in ds:
    t.add_row([d, F(d, c, f, N)])

print(t.draw())

+---+-------+
| d | F     |
+---+-------+
| 1 | 0.823 |
+---+-------+
| 2 | 0.504 |
+---+-------+
| 3 | 0.229 |
+---+-------+
| 4 | 0.080 |
+---+-------+
| 5 | 0.022 |
+---+-------+
| 6 | 0.005 |
+---+-------+


Compute the probability of adversaries being able to predict the beacon of the next `k`th epoch if using the smallest `m` betas from a block with at least `d` backers to generate the beacon

In [39]:
def B0(m, d, c, N):
    sum = 0
    for i in range(d, N):
        sum += binom.pmf(i, N, c/N) * math.comb(i-d+m, m)
    return sum

def B(a, m, d, c, N):
    ret = a / (1 - a)
    ret *= B0(m, d, c, N)
    return ret

def P(a, m, d, c, N):
    ret1 = 0
    ret2 = 0
    f = math.ceil(a * N)
    for i in range(d-m, f):
        if i >= d:
            ret1 += binom.pmf(i, f, c/N) * math.comb(i, m)
        else:
            ret1 += binom.pmf(i, f, c/N) * math.comb(i, d-i) / math.comb(d, d-i)

    for i in range(d, f):
        x = 0
        for j in range(m, i):
            x += binom.pmf(j, f, c/N) * math.comb(j, m) / math.comb(i, m)
        ret2 += binom.pmf(i, N, c/N) * x

    return ret1*a + ret2*(1-a)

def M(a, m, d, c, N):
    p = P(a, m, d, c, N)
    return p / (1 - p) 

def Pd(k, a, m, d, c, N):
    return B(a, m, d, c, N) * M(a, m, d, c, N)**(k-1)

t = Texttable()
t.set_cols_dtype(['i', 'i', 'e', 'e', 'e', 'e', 'e'])
t.add_row(['d', 'm', 'B', 'M', 'Pd_2', 'Pd_3', 'Pd_4'])

a, c, N = 1/5, 8, 101
ms = [1, 2, 3, 4, 5]
ds = [5, 6, 7]

for d in ds:
    for m in ms:     
        t.add_row([d, m, B(a,m,d,c,N), M(a,m,d,c,N), Pd(2,a,m,d,c,N), Pd(3,a,m,d,c,N), Pd(4,a,m,d,c,N)])

print(t.draw())

+---+---+-----------+-----------+-----------+-----------+-----------+
| d | m | B         | M         | Pd_2      | Pd_3      | Pd_4      |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 1 | 1.013e+00 | 2.266e-01 | 2.294e-01 | 5.198e-02 | 1.178e-02 |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 2 | 3.417e+00 | 1.172e-01 | 4.005e-01 | 4.695e-02 | 5.503e-03 |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 3 | 9.863e+00 | 9.258e-02 | 9.131e-01 | 8.453e-02 | 7.825e-03 |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 4 | 2.564e+01 | 6.037e-02 | 1.548e+00 | 9.348e-02 | 5.644e-03 |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 5 | 6.174e+01 | 3.267e-02 | 2.017e+00 | 6.588e-02 | 2.152e-03 |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 6 | 1 | 7.853e-01 | 1.616e-01 | 1.269e-01 | 2.051e-02 | 3.314e-03 |
+---+---+-----------